In [1]:
#importing all the required libraries
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sklearn.metrics
import math
from sklearn.metrics import mean_squared_error

In [2]:
#. Downloading the data
#Read the data for January. How many columns are there

df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df_jan.shape

(3066766, 19)

In [3]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
#Computing duration
#Now let's compute the duration variable. It should contain the duration of a ride in minutes.
#What's the standard deviation of the trips duration in January?

df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

df = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
print(np.std(df))

VendorID                                  0.444035
tpep_pickup_datetime     8 days 17:01:31.093461398
tpep_dropoff_datetime    8 days 17:01:30.682207701
passenger_count                           0.895423
trip_distance                            251.95594
RatecodeID                                6.051628
PULocationID                             64.246082
DOLocationID                             69.891282
payment_type                              0.512936
fare_amount                              16.212411
extra                                     1.783131
mta_tax                                   0.093148
tip_amount                                3.645203
tolls_amount                              1.934029
improvement_surcharge                     0.166954
total_amount                             20.535464
congestion_surcharge                      0.737779
airport_fee                               0.348071
duration                                  9.939384
dtype: object


/home/codespace/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3579: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [5]:
df_jan.shape

(3066766, 20)

In [6]:
df_jan_no_outliers = df.loc[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]
df_jan_no_outliers.shape

(3009173, 20)

In [7]:
3009173 / 3066766 * 100

98.1220282212598

In [8]:
# #One hot encoder
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

/tmp/ipykernel_34216/1257913725.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [9]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.649137551399666

In [10]:
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
df_feb.shape

(2913955, 19)

In [11]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [12]:
df_feb['duration'] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]
print(np.std(df_feb))

VendorID                                  0.446868
tpep_pickup_datetime     9 days 15:24:45.537531357
tpep_dropoff_datetime    9 days 15:24:49.550453395
passenger_count                           0.888353
trip_distance                           270.934631
RatecodeID                                6.185342
PULocationID                             64.136108
DOLocationID                             69.702474
payment_type                                0.5116
fare_amount                              15.805872
extra                                     1.784592
mta_tax                                   0.089713
tip_amount                                3.633154
tolls_amount                              1.926315
improvement_surcharge                      0.16978
total_amount                             20.099554
congestion_surcharge                      0.724938
Airport_fee                               0.330019
duration                                 10.064225
dtype: object


/home/codespace/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3579: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [13]:
df_feb.shape

(2855951, 20)

In [14]:
df_feb_no_outliers = df_feb.loc[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]
df_feb_no_outliers.shape

(2855951, 20)

In [15]:
categorical = ['PULocationID']
numerical = ['trip_distance']

df_feb[categorical] = df_feb[categorical].astype(str)

In [16]:
train_dicts = df_feb[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_feb[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

8.603425051444331